In [ ]:
# Цель бинарной классификации - предсказать,
#подпишется ли клиент на срочный банковский депозит (переменная y).
#...........................................................................
#  [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, In press, http://dx.doi.org/10.1016/j.dss.2014.03.001

 # Available at: [pdf] http://dx.doi.org/10.1016/j.dss.2014.03.001
                #[bib] http://www3.dsi.uminho.pt/pcortez/bib/2014-dss.txt
#............................................................................
import os
print(os.listdir("../input"))

Подключите необходимые библиотеки.

In [ ]:
 import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)}); # you can change this if needed

Загрузите данные. Они находятся в директории input.

In [ ]:
 df = pd.read_csv('../input/bank-additional-full.csv', sep=';')
    # Имя файла (или URL), тип разделителя ячеек (по умолчанию --- запятая), 
    #наличие строки заголовка (указывается её номер; по умолчанию имена признаков считываются из 
    #первой строки файла), наличие колонки с индексами (идентификаторами) 
    #строк (также указывается номер; по умолчанию --- отсутствует)

In [ ]:
df.head()
# Метод head(n) предназначен для просмотра первых n строк таблицы (по умолчанию n=5)

In [ ]:
df.head(101).T
# Если признаков (столбцов) слишком много, полезно будет транспонировать вывод

In [ ]:
df.info()
# Метод info() позволяет вывести общую информацию о датасете
# Мы можем узнать тип каждого признака, а также есть ли в данных пропуски

In [ ]:
sns.boxplot(df['age']); # выбросы (outliers) --- точки, "выбивающиеся" из общей картины. 
#Поэтому полезно применить boxplot ("ящик с усиками").

In [ ]:
def outliers_indices(feature):
    '''
    Будем считать выбросами все точки, выходящие за пределы трёх сигм.
    '''
    mid = df[feature].mean()
    sigma = df[feature].std()
    return df[(df[feature] < mid - 3*sigma) | (df[feature] > mid + 3*sigma)].index

In [ ]:

wrong_age = outliers_indices('age')
out = set(wrong_age)
print(len(out))
# Из набора данных будет удалено 369 объектов-выбросов, что не является существенным в данном случае.

Каков средний возраст холостых/незамужних клиентов?

In [ ]:

df.groupby('marital')['age'].mean() #объединяем стобец со значениями "положение" и "возраст". Находим среднее значение.

In [ ]:
df.groupby('marital')['age'].mean().plot(kind='bar') #объединяем стобец со значениями "положение" и "возраст". Находим среднее значение.
plt.ylabel('Age') # добавляем подпись на оси Оу
plt.show(); #строим график

In [ ]:
plt.figure(figsize=(10, 30)) # увеличим размер картинки
sns.countplot(y='age', hue='marital', data=df); # строим график

В какой день недели (признак **day_of_week**) чаще всего звонили клиентам, отказавшимся от депозита?

In [ ]:
#df['day_of_week'].value_counts() # Считаем кол-во звонков (в какой именно день),
#которые совершались клиентам, отказавшимся от депозита. Можем сделать вывод, что чаще всего звонили в понедельник. 
#Реже - в пятницу. 
#Серди согласившихся такие результаты: чаще всего звонили в четверг, реже в - пятницу.
df.groupby('y')['day_of_week'].value_counts()

In [ ]:
# В процентном получаем результат. Результат подтвердился:в понедельник звонили чаще (среди тех, кто отказался).
df.groupby('y')['day_of_week'].value_counts(normalize=True)

In [ ]:
df.groupby('y')['day_of_week'].value_counts().max() # Данный результат соответсвует понедельнику.

Постройте инфографику по признакам **marital** и **y**. Какие выводы можно сделать?

In [ ]:

plt.figure(figsize=(15, 5)) # увеличим размер картинки
sns.countplot(y='marital', hue='y', data=df); # строим график
# График показывает, что больше всего депозит берут люди, которые находятся в браке. 
#Также категория людей в браке лидирует по отказу от депозита(>20000).
#Вторым "лидером" относится группа одиноких людей.
#Меньше всего берут(не берут) депозит люди, которые не указали свое семейное положение.

In [ ]:
df.groupby('y')['marital'].value_counts().plot(kind='bar')
# График показывает, что больше всего депозит берут(не берут) люди, которые находятся в браке.
#Меньше всего берут(не берут) люди, которые не имеют данных (unknown).

In [ ]:
df.groupby('y')['marital'].value_counts().plot() # Группируем по значениям "семейное положение" и "депозит". Строим график.

In [ ]:
df.groupby('marital')['y'].value_counts().plot(kind='bar') # Группируем по значениям "семейное положение" и "депозит".
plt.ylabel('y') # добавляем подпись на оси Оу
plt.show();

Имеется ли связь между наличием кредита (**default**) у клиента и результатом? Постройте инфографику.

In [ ]:
sns.heatmap(pd.crosstab(df['default'], df['poutcome']), 
            cmap="YlGnBu", annot=True, cbar=False); #выявляем зависимость между наличием кредита "default" и результатом "poutcome"

In [ ]:
# Попробуем вычислить коэффициент 𝜒2
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['default'], df['poutcome'])) 
#Малое значение p-value говорит о том, что связь статистически подтверждается.


In [ ]:
#Рассмотрим взаимосвязь депозита и результата. Построим кросс-таблицу.
pd.crosstab(df['default'], df['poutcome'])

Визуализируйте информацию об уровне образования в зависимости от среднего возраста. Есть ли статистически значимая взаимосвязь между ними?

In [ ]:

df.groupby('education')['age'].mean().plot(kind='bar') # Объединяем значения стобца "Образование" и столбца "Возраст". Подсчитываем средний возраст.   
plt.ylabel('Age') # добавляем подпись на оси Оу
plt.show();
# Можем сделать вывод,что люди в возрасте от 40 до 50 лет составляют группу (illiterate), (basic.4y) и (unknown). 
# Чуть выше барьера в 40 лет превосходит группа (basic.6y).
# Остальные группы людей, которые получили образование (basic.9y),(high.school),(professional.course), (university.degree) в возрасте от 30 до 40. 

In [ ]:
plt.figure(figsize=(15, 8)) # увеличим размер картинки
sns.countplot(y='education', hue='age', data=df);
#  Распределено количество людей (уровень образования) по возрастным группам. 
#Здесь нам поможет график countplot библиотеки Seaborn.

In [ ]:
# Попробуем вычислить коэффициент  𝜒2
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['education'], df['age']))
#Малое значение p-value говорит о том, что связь статистически подтверждается.

Что можно сказать о связи между длительностью контакта (признак **duration**) и возрастом клиента? Найдите коэффициент корреляции.

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
r = pearsonr(df['duration'], df['age'])
print('Pearson correlation:', r[0], 'p-value:', r[1])
# Так как p-value > 0.05 (типичное пороговое значение), 
#то делаем вывод о том, что взаимосвязь (корреляция) между duration(образованием) 
#и age(возрастом) статистически незначима.


Как связаны признаки **education** и **housing**?

In [ ]:
pd.crosstab(df['education'], df['housing']) # Ответим на данный вопрос с помощью кросс-таблицы.
# Данная таблица имеет 3 столбца и 8 строк. Строки отвечают за информацию об образовнии. Столбцы содержат данные, характеризующие наличие, отсутсвие или непринадлежность к двум группам.
#Делаем вывод по данной выборке,что клиенты с (university.degree) имеют имеют место проживания. (6504 - самы высокий показатель)
#У людей безграмотнх(illiterate) самый низкий показатель (10). 
#Самый высокий показатель людей, которые не имеют жилья, составляют клиенты с (university.degree).
#Самый низкий показаьель (8) у клиентов безграмотных(illiterate).
#Люди, закончившие ВУЗ (university.degree), и безграмотные(illiterate) люди лидируют в данной выборке.
#Клиенты, закончившие среднюю школу (high.school), имеют жилье(4932), это второй показатель по "высоте".
#Можем сделать вывод,что признаки education и housing связаны прямолинейно. 

Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

In [ ]:
df.groupby('housing')['pdays'].mean() #объединяем стобец со значениями "жилье" и "количество дней, 
#прошедших с момента последнего обращения к клиенту в рамках предыдущей кампании". 
#Находим среднее значение

In [ ]:
df.groupby('marital')['nr.employed'].value_counts() # Подсчитывваем кол-во сотрудников в зависимости от их семейного положения.

In [ ]:
df.groupby('marital')['nr.employed'].mean().plot(kind='bar') # Объединяем значения стобца "Семейное положение" и столбца "Количество сотрудников ". Подсчитываем средний возраст.   
plt.ylabel('nr.employed') # добавляем подпись на оси Оу
plt.show();

In [ ]:
# Попробуем вычислить коэффициент 𝜒2
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['education'], df['cons.conf.idx'])) 
#Малое значение p-value говорит о том, что связь статистически подтверждается.

In [ ]:
# Рассмотрим взаимосвязь ставки euribor на 3 месяца и коэффициент вариации занятости(квартальный показатель).
pd.crosstab(df['euribor3m'], df['emp.var.rate'])
#Построили кросс-таблицу.